In [1]:
pip install pandas python-binance datetime

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 

with open("../binance.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
    api_secret = lines[0].strip()
    api_key = lines[1].strip()


In [3]:
from binance.client import Client

class Binance_Libs:
    def __init__(self):
        self.api_key = api_key
        self.api_secret = api_secret
        self.client = Client(api_key, api_secret)


In [4]:
from datetime import datetime, timedelta
class Candles:
    def __init__(self, api_client):
        self.client = api_client
    
    async def get_stochastic_klines(self, symbol, kline_list, window=14, K_length=5, K_smoothing=5, D_smoothing=5):
        try:
            start_time = datetime.now() - timedelta(days=100)
            start_time_timestamp = int(start_time.timestamp()) * 1000
    
            loop = asyncio.get_event_loop()
            klines = await loop.run_in_executor(None, self.client.get_historical_klines, symbol,
                                                        self.client.KLINE_INTERVAL_1HOUR,
                                                str(start_time_timestamp), str(datetime.now()))
            
            close_prices = [float(kline[4]) for kline in klines]
            highest_high = [float(kline[2]) for kline in klines]
            lowest_low = [float(kline[3]) for kline in klines]

            if klines:
                kline_list.append([symbol, close_prices, highest_high, lowest_low])

            return kline_list
    
        except Exception as e:
            print(f"{symbol}: Hata oluştu {e}")

In [5]:
binance_libs = Binance_Libs()
candles_instance = Candles(binance_libs.client)

info = binance_libs.client.futures_exchange_info()
symbols_usdt = [symbol['symbol'] for symbol in info['symbols'] if symbol['symbol'].endswith('USDT')]

print("BBR", symbols_usdt)

BBR ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT', 'FLMUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'OCEANUSDT', 'CVCUSDT', 'BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'UNFIUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT', 'BTCSTUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT',

In [6]:
import asyncio

kline_list = []

results = await asyncio.gather(*(candles_instance.get_stochastic_klines(symbol, kline_list) for symbol in symbols_usdt))


DEFIUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000SHIBUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
BTCDOMUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000XECUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000LUNCUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
LUNA2USDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000PEPEUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000FLOKIUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
DODOXUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
BIGTIMEUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
BSVUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
ORBSUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
TOKENUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
KASUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000BONKUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
ETHWUSDT: Hata oluştu APIError(code=-1121): Invalid symbol.
1000RATSUS

In [7]:
import pandas as pd

for kline_data in kline_list:
    symbol = kline_data[0]
    close_price = kline_data[1]
    high_price = kline_data[2]
    low_price = kline_data[3]

    # DataFrame oluşturma
    data = pd.DataFrame({
        'Symbol': [symbol],
        'Close': [close_price],
        'High': [high_price],
        'Low': [low_price]
    })


    data.to_parquet(f"data/{symbol}.parquet", index = False) 
    data.to_pickle(f"data/{symbol}.pkl")

In [10]:
data = pd.read_pickle("data/ETHUSDT.pkl")
print(data)

    Symbol                                              Close  \
0  ETHUSDT  [2576.6, 2580.03, 2581.56, 2580.93, 2571.6, 25...   

                                                High  \
0  [2611.52, 2582.75, 2588.8, 2587.56, 2583.37, 2...   

                                                 Low  
0  [2571.57, 2563.73, 2578.22, 2578.0, 2556.8, 25...  
